In [259]:
import pandas as pd
import numpy as np
import json
import os
import tqdm
import re
import xgboost as xgb
from sklearn import preprocessing
from sklearn import cross_validation 

In [312]:
def extract_date(row):
    return row['postedDate']['$date']

def get_df_from_json(filename, mode):    
    with open(filename) as json_data:
        data = json_data.readlines()
        data = map(lambda x: x.rstrip(), data)
        data_json_str = "[" + ','.join(data) + "]"
        df = pd.read_json(data_json_str)
        if mode != 'test':
            df = df.drop('_id', axis = 1)
            df.postedDate = df.apply(lambda row: extract_date(row), axis = 1)
        return df

df = get_df_from_json('jobs_huge.json', mode = 'train')

In [313]:
new_df = pd.DataFrame(columns=['string', 'class'])
columns = df.columns

#wanted to name it extract
def extra_string_n_class(row, new_df):
    for column in tqdm.tqdm(columns):
        new_df.loc[len(new_df)]=[row[column], column] 
        

df.apply(lambda row: extra_string_n_class(row, new_df), axis = 1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
3066    None
3067    None
3068    None
3069    None
3070    None
3071    None
3072    None
3073    None
3074    None
3075    None
3076    None
3077    None
3078    None
3079    None
3080    None
3081    None
3082    None
3083    None
3084    None
3085    None
3086    None
3087    None
3088    None
3089    None
3090    None
3091    None
3092    None
3093    None
3094    None
3095    None
dtype: object

In [314]:
# to save time if my ipython starts again.
new_df.to_csv('transformed_jobs.csv', encoding='utf-8', index=False)
new_df = pd.read_csv('transformed_jobs.csv', encoding='utf-8')

In [333]:
######Feature Engineering#######
def all_digits(key):
    try:
        x = int(key)
    except ValueError: 
        return 0
    return 1

def num_digits(key):
    try:
        count = sum(c.isdigit() for c in key)
        return count
    except TypeError:
        print "error while counting digts in", key
        return 10

def has_html_tags(key):
    try:
        pat = re.compile('<.*?>')
        match = re.match(pat, key)
        if match:
            return 1
        else:
            return 0
    except TypeError:
        print "error while has_html_tags  in", key
        return 10
    
def len_str(key):
    return len(key)

def occurance_count(df, key, keyname):
    return len(df[df[keyname] == key])

In [316]:
#save occurance as feature and then drop duplicates
new_df['occurance_count'] = new_df.apply(lambda row: occurance_count(new_df, row['string'], 'string'), axis = 1)
new_df = new_df.drop_duplicates()

In [317]:
# New feature columns 'all_digits', 'num_digits', 'has_html_tags', 'len_str', 'is_known_country', 'occurance_count'
new_df['all_digits'] = new_df.apply(lambda row: all_digits(row['string']), axis = 1)
new_df['num_digits'] = new_df.apply(lambda row: num_digits(row['string']), axis = 1)

In [318]:
new_df['has_html_tags'] = new_df.apply(lambda row: has_html_tags(row['string']), axis = 1)
new_df['len_str'] = new_df.apply(lambda row: len_str(row['string']), axis = 1)

In [319]:
###########Classification############
le_class = preprocessing.LabelEncoder()
le_class.fit(new_df['class'])
print le_class.classes_
new_df['en_class'] = le_class.transform(new_df['class'])
new_df

[u'category' u'city' u'country' u'description' u'postedDate' u'state'
 u'title']


,string,class,occurance_count,all_digits,num_digits,has_html_tags,len_str,en_class
0,Management Consulting,category,1590,0,0,0,21,0
1,Chicago,city,301,0,0,0,7,1
2,United States,country,3096,0,0,0,13,2
3,<DIV><B>SAP Package Technologies</B>: Perform ...,description,1,0,77,1,7792,3
4,1454025600000,postedDate,72,1,13,0,13,4
5,Illinois,state,301,0,0,0,8,5
6,SAP Ariba - Senior Consultant,title,5,0,0,0,29,6
8,Denver,city,118,0,0,0,6,1
10,<DIV><DIV><STRONG>Application Management Servi...,description,1,0,195,1,15942,3
12,Colorado,state,119,0,0,0,8,5


In [320]:
Y = new_df.en_class
X = new_df.drop(['string','class', 'en_class'], axis = 1)

In [321]:
Y = Y.reshape(6048, 1)

In [322]:
clf = xgb.XGBClassifier(objective='reg:logistic', nthread=4, seed=0)  

clf.fit(X,Y)


/home/amol/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [323]:
Y = Y.reshape(6048,)
#by default 3 fold cross_validation
scores = cross_validation.cross_val_score(clf, X, Y)

In [324]:
print scores

array([ 0.98266469,  0.99156328,  0.98857994])

In [325]:
clf.score(X,Y)
#accurcy 99% on training set

0.99388227513227512

In [382]:
test_df = get_df_from_json('test/test_tiny_1.txt', mode = 'test')

In [383]:
test_new_df = test_df[test_df['key']!='']
#5112 non-empty keys, 882 empty keys.

In [384]:
# to save time if my ipython starts again.
test_new_df.to_csv('transformed_test.csv', encoding='utf-8', index=False)
test_new_df = pd.read_csv('transformed_test.csv', encoding='utf-8')

test_new_df['occurance_count'] = test_new_df.apply(lambda row: occurance_count(test_new_df, row['key'], 'key'), axis = 1)
test_new_df = test_new_df.drop_duplicates()

strings = test_new_df['key']
# New feature columns 'all_digits', 'num_digits', 'has_html_tags', 'len_str', 'is_known_country', 'occurance_count'
test_new_df['all_digits'] = test_new_df.apply(lambda row: all_digits(row['key']), axis = 1)
test_new_df['num_digits'] = test_new_df.apply(lambda row: num_digits(row['key']), axis = 1)

test_new_df['has_html_tags'] = test_new_df.apply(lambda row: has_html_tags(row['key']), axis = 1)
test_new_df['len_str'] = test_new_df.apply(lambda row: len_str(row['key']), axis = 1)

test_new_df

,actual,id,key,occurance_count,all_digits,num_digits,has_html_tags,len_str
0,NaN,54ae5da6e4b08621addd287a,Senior Pricing and Business Intelligence Analyst,1,0,0,0,48
1,NaN,54ae5da6e4b08621addd287a,Marketing,18,0,0,0,9
2,NaN,54ae5da6e4b08621addd287a,"<h2 class=""iCIMS_SubHeader iCIMS_SubHeader_Job...",1,0,35,1,4961
3,NaN,54ae5da6e4b08621addd287a,Malvern,12,0,0,0,7
4,NaN,54ae5da6e4b08621addd287a,PA,91,0,0,0,2
5,NaN,54ae5da6e4b08621addd287d,Property Accountant,1,0,0,0,19
6,NaN,54ae5da6e4b08621addd287d,Accounting/Finance,1,0,0,0,18
7,NaN,54ae5da6e4b08621addd287d,"<h2 class=""iCIMS_SubHeader iCIMS_SubHeader_Job...",1,0,20,1,3615
8,NaN,54ae5da6e4b08621addd287d,Malvern,12,0,0,0,7
9,NaN,54ae5da6e4b08621addd287d,PA,91,0,0,0,2


In [385]:
id = test_new_df.id

In [386]:
X = test_new_df.drop(['actual', 'id', 'key'], axis = 1)

In [387]:
Y_predict = clf.predict(X)

In [388]:
Y_predict = le_class.inverse_transform(Y_predict)

In [389]:
Y_predict

array([u'title', u'city', u'description', ..., u'state', u'state',
       u'category'], dtype=object)

In [390]:
#dropped empty keys and dropped duplicates.
print len(id), len(Y_predict), len(strings)

51723 51723 51723


In [391]:
ans_df = pd.DataFrame({'id': id, 'actual': Y_predict, 'key': strings})
ans_df.to_csv('test_tiny_1_out.csv', index= False, encoding='utf=8')

In [392]:
ans_df

,actual,id,key
0,title,54ae5da6e4b08621addd287a,Senior Pricing and Business Intelligence Analyst
1,city,54ae5da6e4b08621addd287a,Marketing
2,description,54ae5da6e4b08621addd287a,"<h2 class=""iCIMS_SubHeader iCIMS_SubHeader_Job..."
3,city,54ae5da6e4b08621addd287a,Malvern
4,state,54ae5da6e4b08621addd287a,PA
5,title,54ae5da6e4b08621addd287d,Property Accountant
6,title,54ae5da6e4b08621addd287d,Accounting/Finance
7,description,54ae5da6e4b08621addd287d,"<h2 class=""iCIMS_SubHeader iCIMS_SubHeader_Job..."
8,city,54ae5da6e4b08621addd287d,Malvern
9,state,54ae5da6e4b08621addd287d,PA
